In [21]:
64*3


192

In [1]:
import tensorflow as tf
import numpy as np

ImageSize = 448
NumChannel = 3
NumClass = 1000

BatchSize = 4
FilterSize1 = 7
OutputSize1 = 64*NumChannel
NumStrides = 2

FilterSize2 = 3
OutputSize2 = 256

F2 = tf.Variable(tf.truncated_normal(shape=[FilterSize2,FilterSize2,OutputSize1,OutputSize2], stddev=0.1))

weights = {
    # 7x7 conv, 3 input, 192 outputs
    'F1': tf.Variable(tf.truncated_normal(shape=[FilterSize1,FilterSize1,NumChannel,OutputSize1], stddev=0.1)),
    # 3x3 conv, 192 inputs, 256 outputs
    'F2': tf.Variable(tf.truncated_normal(shape=[FilterSize2,FilterSize2,OutputSize1,OutputSize2], stddev=0.1)),
    # 3x3 conv, 256 inputs, 512 outputs
    'F3': tf.Variable(tf.truncated_normal(shape=[3,3,256,512], stddev=0.1)),
    # 3x3 conv, 512 inputs, 1024 outputs
    'F4': tf.Variable(tf.truncated_normal(shape=[3,3,512,1024], stddev=0.1)),
    # 3x3 conv, 1024 inputs, 1024 outputs
    'F5': tf.Variable(tf.truncated_normal(shape=[3,3,1024,1024], stddev=0.1)),
    # fully connected, 7*7*1024 inputs, 4096 outputs
    'W6': tf.Variable(tf.random_normal([7*7*1024, 4096])),
    
    
    
}

biases = {
    'bc1': tf.Variable(tf.random_normal([OutputSize1])),
    'bc2': tf.Variable(tf.random_normal([OutputSize2])),
    'bc3': tf.Variable(tf.random_normal([512])),
    'bc4': tf.Variable(tf.random_normal([1024])),
    'bc5': tf.Variable(tf.random_normal([1024])),
    'bw6': tf.Variable(tf.random_normal([4096])),
    
}
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [2]:
# Fast YOLO with 9 layers
input_image = tf.placeholder(tf.float32,shape=(BatchSize,ImageSize,ImageSize,NumChannel))
output = tf.placeholder(tf.float32, shape=(BatchSize, NumClass))


# Conv Layer 1

strides = [1,NumStrides,NumStrides,1]
conv1 = conv2d(x=input_image, W=weights['F1'], b=biases['bc1'], strides=NumStrides)
mxpl1 = tf.nn.max_pool(conv1, ksize=(1,2,2,1), strides=strides,padding='SAME')

print('conv1: ',conv1.shape)
print('mxpl1: ', mxpl1.shape)

# Conv Layer 2
conv2 = conv2d(x=mxpl1, W=weights['F2'], b=biases['bc2'],strides=1)
mxpl2 = tf.nn.max_pool(conv2, ksize=(1,2,2,1), strides=strides,padding='SAME')

print('conv2: ',conv2.shape)
print('mxpl2: ', mxpl2.shape)

# Conv Layer 3
conv3 = conv2d(x=mxpl2, W=weights['F3'], b=biases['bc3'],strides=1)
mxpl3 = tf.nn.max_pool(conv3, ksize=(1,2,2,1), strides=strides,padding='SAME')

print('conv3: ',conv3.shape)
print('mxpl3: ', mxpl3.shape)

# Conv Layer 4
conv4 = conv2d(x=mxpl3, W=weights['F4'], b=biases['bc4'],strides=1)
mxpl4 = tf.nn.max_pool(conv4, ksize=(1,2,2,1), strides=strides,padding='SAME')

print('conv4: ',conv4.shape)
print('mxpl4: ', mxpl4.shape)

# Conv Layer 5
conv5 = conv2d(x=mxpl4, W=weights['F5'], b=biases['bc5'],strides=1)
mxpl5 = tf.nn.max_pool(conv5, ksize=(1,2,2,1), strides=strides,padding='SAME')

print('conv5: ',conv5.shape)
print('mxpl5: ', mxpl5.shape)
# Fully connected 6
ful6 = tf.reshape(mxpl5,[-1, 7*7*1024])
ful6 = tf.layers.dense(inputs=ful6,units=4096,activation=tf.nn.relu)
ful7 = tf.layers.dense(inputs=ful6,units=7*7*30,activation=tf.nn.relu)
ful7 = tf.reshape(ful7,[-1,7,7,30])

print('ful7: ',ful7.shape)



conv1:  (4, 224, 224, 192)
mxpl1:  (4, 112, 112, 192)
conv2:  (4, 112, 112, 256)
mxpl2:  (4, 56, 56, 256)
conv3:  (4, 56, 56, 512)
mxpl3:  (4, 28, 28, 512)
conv4:  (4, 28, 28, 1024)
mxpl4:  (4, 14, 14, 1024)
conv5:  (4, 14, 14, 1024)
mxpl5:  (4, 7, 7, 1024)
ful7:  (4, 7, 7, 30)
